#Innomatics Reasearch Labs-Advanced GenAI Internship

##Food Delivery Data Analysis Hackathon

Objective:

-Merge transactional, user, and restaurant datasets

-Analyze revenue, user behaviour, cuisine trends, and seasonality

In [ ]:
#Importing the libraries
import pandas as pd 
import numpy as np 
import sqlite3

In [ ]:
#Load orders.csv
orders=pd.read_csv("C:\\Users\\manda\\OneDrive\\Desktop\\Test\\orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
#Load json of users
users=pd.read_json("C:\\Users\\manda\\OneDrive\\Desktop\\Test\\users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [8]:
# Create / connect to database
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Load SQL file
with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

print("SQL file loaded successfully")


SQL file loaded successfully


In [9]:
pd.read_sql("""
SELECT name FROM sqlite_master 
WHERE type='table'
""", conn)


,name
0,restaurants


In [10]:
# Load restaurant table
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [11]:
#Merge the datasets(left join)
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
#save Final dataset
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [13]:
#city with highest revenue from Gold members
gold_city_revenue = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [14]:
#Cuisine with highest average order value
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [15]:
#Users with total spend > ₹1000
user_spend = df.groupby("user_id")["total_amount"].sum()
count = (user_spend > 1000).sum()
count


np.int64(2544)

In [16]:
#Rating range with highest revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [17]:
#Gold members – highest AOV city
df[df["membership"] == "Gold"] \
  .groupby("city")["total_amount"].mean() \
  .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [18]:
#Cuisine with few restaurants but high revenue
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()

pd.concat([restaurant_count, revenue], axis=1)


,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [19]:
#% orders by Gold members
gold_orders = df[df["membership"] == "Gold"].shape[0]
total_orders = df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [23]:
#Restaurant with highest average order value but less than 20 total orders
restaurant_stats = df.groupby("name").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)
small_restaurants = restaurant_stats[restaurant_stats["total_orders"] < 20]
top_restaurant = small_restaurants.sort_values("avg_order_value", ascending=False)

top_restaurant.head(1)

,avg_order_value,total_orders
name,,
User_2429,1497.42,1


In [21]:
#Highest revenue combination
df.groupby(["membership", "cuisine"])["total_amount"].sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
#Highest revenue quarter
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


C:\Users\manda\AppData\Local\Temp\ipykernel_4100\3990236579.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [25]:
#Total orders placed by users with Gold membership
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
gold_orders_count 

4987

In [26]:
#Total revenue generated from orders placed in Hyderabad city
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)

1889367

In [27]:
#Number of distinct users who placed at least one order
distinct_users = df['user_id'].nunique()
distinct_users

2883

In [28]:
#Average order value for Gold members
gold_avg_order_value = df[df['membership'] == 'Gold']['total_amount'].mean()
round(gold_avg_order_value, 2) 

np.float64(797.15)

In [29]:
#Number of orders placed for restaurants with rating >= 4.5
high_rating_orders = df[df['rating'] >= 4.5].shape[0]
high_rating_orders

3374

In [30]:
#Number of orders placed in the top revenue city among Gold members
top_gold_city = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()
top_gold_city_orders = df[(df['membership'] == 'Gold') & (df['city'] == top_gold_city)].shape[0]
top_gold_city_orders

1337

In [ ]:
#Total number of rows in the final merged dataset
total_rows = df.shape[0]
total_rows  


10000